<a href="https://colab.research.google.com/github/ashishk121/Quantum-Hamming-distance-for-Annealer/blob/main/QUBO_formulation_quantum_hamming_distance.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [179]:
import numpy as np
import sympy as sp
from scipy import linalg

In [180]:
# n is the size of the matrix B which defines the real self dual code
# d is the total size of the matrix in QUBO ready form 
n=int(6);
d=int(n + pow(n,2)/2)

In [181]:
B=np.array([[0,1,1,1,1,1],[1,0,1,0,0,1],[1,1,0,1,0,0],[1,0,1,0,1,0],[1,0,0,1,0,1],[1,1,0,0,1,0]]);

In [183]:
I=np.identity(n);
Bsq= B.dot(B);
Bsqd=np.diagonal(Bsq)
# This is Generator matrix for the code
G= np.concatenate((I,B),axis=1) 

In [184]:
# Initialize to 0
M=np.zeros((d,d))
# Fill out top-left block
for i in range(0,n):
  for j in range(0,n):
    if j>i:
      M[i,j]=-2*B[i,j]+ 2*Bsq[i,j]
    elif i == j:
      M[i,j]= 1+Bsqd[i]
    else:
      M[i,j]=0

# Fill out first n rows
for p in range(2,n-1):
  
  for i in range(0,n):
    for j in range((p-1)*n,p*n):
      M[i,j]=2*I[i,j-(p-1)*n]-4*B[i,j-(p-1)*n]

#Fill out the main diagonal
for i in range(n,d):
  for j in range(0,d):
    if i==j:
      M[i,j]=4

#Fill out the parallel diagonals
for q in range(1,int(n/2)):
  for i in range(n,d-n):
    if i+q*n<d:
      M[i,i+q*n]=8                   

In [185]:
# In order to rule out the trivial codeword, we prepare n matrices of size d-1. These are elements of list
# The true min is  min ( initial[i] + QUBO min of list[i])

list=[];
initial=[];
v00=np.zeros((n,n-1))
v0=np.zeros((n,n))
for i in range(0,n):
   for j in range(0,n):
     if i<j:
       v0[i,j]=M[i,j]
     elif j<i:
       v0[i,j]=M[j,i]
   v00[i]=np.delete(v0[i],i)    
      
   v1=M[i][n:]
   v2=np.concatenate((v00[i],v1))
   v3=np.diag(v2)
   v4=np.delete(np.delete(M,i,0),i,1)+v3
   list.append(v4)
   initial.append(M[i,i])